<a href="https://colab.research.google.com/github/mmassonn/brain_tumor_classification/blob/main/brain_tumor_classification_Xception_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from keras.utils import np_utils

In [ ]:
#Connect drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Load the training dataset

In [ ]:
def loading_data(pathway):
  
  X_data=[]
  y_data=[]

  data_dir = pathway
  categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

  for i in categories:
      path = os.path.join(data_dir,i)
      for j in os.listdir(path):
          img = cv2.imread(os.path.join(path,j), cv2.COLOR_GRAY2RGB)

          X_data.append(img)
          y_data.append(i)
  
  return X_data, y_data

In [ ]:
X_train, y_train = loading_data('drive/MyDrive/Colab Notebooks/BTC/Training')

##Data preparation

In [ ]:
def resize_img(X_train):
  
  X_train_new=[]
  
  for img in X_train:
    img = cv2.resize(img,(256,256))
    X_train_new.append(img)
  
  X_train = X_train_new
  
  return X_train

In [ ]:
def array_img(X_train):

  X_train=np.array(X_train)

  return X_train

In [ ]:
def reshape_img(X_train):

  X_train = X_train.reshape(-1, 256, 256, 3)
  
  return X_train

In [ ]:
X_train = resize_img(X_train)
X_train = array_img(X_train)
X_train = reshape_img(X_train)

##Split training dataset and validation dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)   

In [ ]:
def convert_data(data):

  new_data = []
  categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

  for i in data:
      new_data.append(categories.index(i))

  data = tf.keras.utils.to_categorical(new_data)

  return data

In [ ]:
y_train = convert_data(y_train)
y_val = convert_data(y_val)

##Data augmentation

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

##Model Architecture

###base model with pre-trained weights

In [ ]:
base_model = keras.applications.Xception( weights='imagenet', input_shape=(256, 256, 3), include_top=False)

83697664/83683744 [==============================] - 1s 0us/step


###freeze the base model

In [ ]:
base_model.trainable = False

###Create a new model on top

In [ ]:
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)

scale_layer = layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x) 

outputs = layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

##Configuring the model for training

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer= 'adam',
              metrics=["accuracy"])

##Fit the model

In [ ]:
model.fit( X_train, y_train, epochs=20, validation_data=(X_val, y_val))

##Do a round of fine-tuning of the entire model

In [ ]:
base_model.trainable = True

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer= keras.optimizers.Adam(1e-5),
              metrics=["accuracy"])

In [ ]:
callbacks = [ keras.callbacks.ModelCheckpoint( filepath="convnet_from_scratch.keras", save_best_only=True, monitor="val_loss")]

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=callbacks)

##Displaying curves of loss and accuracy


In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

##Evaluating the model on the test set

In [ ]:
X_test, y_test = loading_data('drive/MyDrive/Colab Notebooks/BTC/Testing')
X_test = resize_img(X_test)
X_test = array_img(X_test)
X_test = reshape_img(X_test)
y_test = convert_data(y_test)

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(X_test, y_test)
test_acc = test_acc * 100
print(f"Test accuracy: {test_acc} %")

13/13 [==============================] - 10s 705ms/step - loss: 1.5720 - accuracy: 0.7589
Test accuracy: 75.88832378387451 %
